# Load in data
of form:
"Time","V1","V2","V3","V4", <br>
"V5","V6","V7","V8","V9", <br>
"V10","V11","V12","V13","V14", <br>
"V15","V16","V17","V18","V19", <br>
"V20","V21","V22","V23","V24", <br>
"V25","V26","V27","V28","Amount","Class"

Where time = seconds since first transaction <br>
V1-V28 = PCA features <br>
Amount = Size of transaction <br>
Class = final classification (fraud vs not)